In [ ]:
%scala
val storage_container = "data"
val storage_account_name = "<NAME>"
val storage_account_access_key = "<KEY>"
val storage_base_url = s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net"

val file_type = "txt"
val train_data_url = s"${storage_base_url}/data/train/**/*.${file_type}"
val  test_data_url = s"${storage_base_url}/data/test/**/*.${file_type}"

storage_container: String = data
storage_account_name: String = cs1100320004e0010e6
storage_account_access_key: String = an5UdS00UuMcb6vT54s2kWCF31rW++P9BKK7K0czqJDmONenltWmfatssB+4tAQOAHbYOUALp2RB+AStHj88Gg==
storage_base_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net
file_type: String = txt
train_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/train/**/*.txt
test_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/test/**/*.txt

In [ ]:
%scala
import org.apache.spark.sql.functions.{col, when, split, size, lower, regexp_replace, input_file_name}
import org.apache.spark.sql.SparkSession

val spark = ( SparkSession.builder()
                          .appName("Pre-Processing-1")
                          .config(s"fs.azure.account.key.${storage_account_name}.blob.core.windows.net", storage_account_access_key)
                          .getOrCreate() )
val sc = spark.sparkContext

import org.apache.spark.sql.functions.{col, when, split, size, lower, regexp_replace, input_file_name}
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2ca398b6
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@2cb4dbcf

In [ ]:
%scala
val trainDF = spark.read.text(train_data_url)
                        .withColumnRenamed("value", "comment")
                        .withColumn("fullPathFileName", input_file_name())
                        .withColumn("splitedFileName", split( col("fullPathFileName"), "/" ) )
                        .withColumn("fileName", col("splitedFileName")( size(col("splitedFileName")) - 1 ) )
                        .select(
                          split( split(col("fileName"), "\\.")(0), "_" )(0).cast("int").as("id"),
                          split( split(col("fileName"), "\\.")(0), "_" )(1).cast("int").as("rating"),
                          col("splitedFileName")( size(col("splitedFileName")) - 2 ).cast("string").as("type"),
                          regexp_replace( lower( col("comment") ), "<[^>]*>", "" ).cast("string").as("comment")
                        )
                        .withColumn(
                          "type",
                           when( col("type") === "pos", 1 )
                          .when( col("type") === "neg", 0 )
                          .otherwise(-1)
                        )
                        .repartition(1)
                        .cache()

val testDF =  spark.read.text(test_data_url)
                        .withColumnRenamed("value", "comment")
                        .withColumn("fullPathFileName", input_file_name())
                        .withColumn("splitedFileName", split( col("fullPathFileName"), "/" ) )
                        .withColumn("fileName", col("splitedFileName")( size(col("splitedFileName")) - 1 ) )
                        .select(
                          split( split(col("fileName"), "\\.")(0), "_" )(0).cast("int").as("id"),
                          split( split(col("fileName"), "\\.")(0), "_" )(1).cast("int").as("rating"),
                          col("splitedFileName")( size(col("splitedFileName")) - 2 ).cast("string").as("type"),
                          regexp_replace( lower( col("comment") ), "<[^>]*>", "" ).cast("string").as("comment")
                        )
                        .withColumn(
                          "type",
                           when( col("type") === "pos", 1 )
                          .when( col("type") === "neg", 0 )
                          .otherwise(-1)
                        )
                        .repartition(1)
                        .cache()

trainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]
testDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]

In [ ]:
%scala
trainDF.show(10)

+-----+------+----+--------------------+
 id|rating|type| comment|
+-----+------+----+--------------------+
16088| 0| -1|***spoilers*** **...|
 1175| 9| 1|match 1: tag team...|
 4383| 9| 1|**attention spoil...|
10044| 9| 1|titanic directed ...|
 4076| 10| 1|by now you've pro...|
 2662| 10| 1|*!!- spoilers - !...|
40647| 0| -1|no no no no.i foo...|
19759| 0| -1|for all its' uber...|
 1930| 4| 0|some have praised...|
 1929| 4| 0|some have praised...|
+-----+------+----+--------------------+
only showing top 10 rows

In [ ]:
%scala
testDF.show(10)

+-----+------+----+--------------------+
 id|rating|type| comment|
+-----+------+----+--------------------+
 8647| 10| 1|there's a sign on...|
 3801| 7| 1|(some spoilers in...|
 4051| 10| 1|back in the mid/l...|
 754| 9| 1|if anyone ever as...|
 3958| 9| 1|films belonging t...|
 9964| 8| 1|the merchant of v...|
 9974| 8| 1|'the merchant of ...|
 5719| 2| 0|possible spoilers...|
10660| 10| 1|the silent film m...|
 4845| 10| 1|without question,...|
+-----+------+----+--------------------+
only showing top 10 rows

In [ ]:
%scala
trainDF.write
       .mode("overwrite")
       .option("header", "true")
       .format("com.databricks.spark.csv")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/train/train_data")

testDF.write
      .mode("overwrite")
      .option("header", "true")
      .format("com.databricks.spark.csv")
      .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/test/test_data")

In [ ]:
%scala
trainDF.write
       .mode("overwrite")
       .format("parquet")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/train/train_data")

testDF.write
      .mode("overwrite")
      .format("parquet")
      .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/test/test_data")